### 
Under stratified random sampling, give a zCDP CI for population mean by adding noise to stratum sums. The sample sizes are considered public. 


In [ ]:
def make_map_partition_meas(meas):
    return Measurement(
        ProductDomain(m.input_domain for m in meas),
        ProductDomain(m.output_domain for m in meas),
        function(|data: Vec<DI::Carrier>|{
            output = []
            for part, m in zip(data, meas):
                output.append(m(part))
            return output
            
        }),
        ProductMetric(meas[0].input_metric),
        meas[0].output_measure, 
        Privacy_Map(|d_in: MI::Distance|{
            return max(m.map(d_in) for m in meas)
        })
    )                  

In [ ]:
def make_map_partition_trans(trans: Transformation):
    return Transformation(
        ProductDomain(t.input_domain for t in trans),
        ProductDomain(t.output_domain for t in trans),
        function(|data: Vec<DI::Carrier>|{
            output = []
            for part, t in zip(data, trans):
                output.append(t(part))
            return output
            
        }),
        ProductMetric(trans.input_metric),
        ProductMetric(trans.output_metric),
        StabilityMap(|d_in: MI::Distance|{
            return max(t.map(d_in) for t in trans)
        })
    )                  

In [2]:
'''
    :param strat_sizes: the population size of each stratum
    :param samp_sizes: sample sizes in each stratum
    :param scale: scale of gaussian noise
    :return private estimators for mean and variance. '''

def postprocess_noisestra_pubsize(samp_sizes, strat_sizes, scale):
    return make_postprocess(
        ProductDomain<AllDomain<f64>>,
        PairDomain<AllDomain<f64>, AllDomain<f64>>
        function(|noisy_sums: Vec<f64> |{
            weights = strat_sizes / sum(strat_sizes)
            noisy_strat_mean = noisy_sums/sample_sizes
            noisy_mean = sum(weights * noisy_stra_mean)
            noisy_strta_var = (strat_sizes - samp_sizes) / strat_sizes * (noisy_strat_mean * (1 - noisy_strat_mean) + scale / samp_sizes**2) / (samp_sizes-1) + scale / n**2
            noisy_var = sum(weights * noisy_strat_var)
            return noisy_mean, noisy_var
        })
    )

SyntaxError: invalid syntax (285232835.py, line 9)

In [ ]:
def make_alg_noisestra_pubsize(samp_sizes, strat_sizes, scale):
    meas_sums = make_map_partition_meas([
        make_sized_bound_sum(n,(0,1)) >> lipschitz_cast((0, n), TI = int, TO = float) >> make_base_gaussian(scale)
        for n in samp_sizes])
    return meas_sums >> postprocess_noisestra_pubsize(samp_sizes, strat_sizes, scale)